In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.rnn.python.ops import rnn, rnn_cell
from tensorflow.contrib.rnn import BasicLSTMCell, static_rnn
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)

hm_epochs = 10
n_classes = 10
batch_size = 128
chunk_size = 28
n_chunks = 28
rnn_size = 128

x = tf.placeholder('float', [None, n_chunks,chunk_size])
y = tf.placeholder('float')

def recurrent_neural_network(x):
    layer = {'weights':tf.Variable(tf.random_normal([rnn_size, n_classes])),
             'biases':tf.Variable(tf.random_normal([n_classes]))}

    x = tf.transpose(x, [1, 0, 2])
    x = tf.reshape(x, [-1, chunk_size])
    x = tf.split(x, n_chunks, 0)

    lstm = BasicLSTMCell(rnn_size, state_is_tuple=True,reuse=True)
    (outputs, states) = static_rnn(lstm, x, dtype=tf.float32)

    output = tf.matmul(outputs[-1], layer['weights']) + layer['biases']

    return output

def train_neural_network(x):
    prediction = recurrent_neural_network(x)
    cost = tf.nn.softmax_cross_entropy_with_logits( logits=prediction, labels=y)
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape((batch_size,n_chunks,chunk_size))

                _, c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_chunks, chunk_size)), y:mnist.test.labels}))

train_neural_network(x)


Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 10 loss: [163.30713 183.53133 179.40483 190.69286 176.69325 175.13876 222.83548
 188.98155 183.91153 187.64864 195.34601 182.70192 216.25853 178.71904
 180.30069 194.9237  201.43462 188.50069 152.39937 192.02057 174.64494
 142.86894 159.21674 189.69888 150.15729 173.87595 204.22983 200.74977
 168.84302 211.67165 193.0477  177.14389 182.38614 183.54906 226.03032
 201.18263 175.49786 208.99101 208.24515 171.47362 183.9878  183.80222
 196.1519  156.99098 194.53993 181.66411 197.57687 185.8486  195.5797
 156.36449 170.60425 169.28429 195.30405 201.27342 169.2541  177.87119
 156.94505 188.71115 166.04561 182.15974 170.97328 193.71866 152.1817
 178.92134 200.52786 162.40868 170.14238 164.87366 163.9144  175.